In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ast


In [ ]:
def plot_subset(f_saples):
    
    samples = pd.read_csv(f_samples, sep="\t")
    merge = samples.merge(genotypes, left_on="IID", right_on="person_id", how="inner")
    plt.scatter(merge['geno'], merge['phenotype'])
    plt.title(f_samples)
    return plt

In [ ]:
genotypes = pd.read_csv("rs11168249genotypes.csv")
genotypes = genotypes[genotypes['geno'] != -9]

In [ ]:
files = ['AFR_BLACK_eur_0', 'AFR_BLACK_eur_1', 'AFR_BLACK_eur_2', 
         'NOT_AFR_BLACK_eur_0', 'NOT_AFR_BLACK_eur_1', 'NOT_AFR_BLACK_eur_2', 
         'passing_samples_v7_eur_0', 'passing_samples_v7_eur_1', 'passing_samples_v7_eur_2']

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 10))
axs = axs.flatten()
for i, file in enumerate(files):
    samples = pd.read_csv(file, sep="\t")
    merge = samples.merge(genotypes, left_on="IID", right_on="person_id", how="inner")
    
    sns.boxplot(x='geno', y='phenotype', data=merge, ax=axs[i])

    axs[i].set_title(file)  # Set title for each subplot
    #axs[i].scatter(merge['geno'], merge['phenotype']) # Example plot (replace with your actual plot)
    
# Hide any unused subplots (if there are less than 9 files)
for j in range(i + 1, len(axs)):
    axs[j].axis('off')  # Turn off axes for unused subplots

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
files = ['AFR_BLACK_platelet.tsv', 'NOT_AFR_BLACK_platelet.tsv', 'passing_samples_v7_platelet.tsv']

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
for i, file in enumerate(files):
    samples = pd.read_csv(file, sep="\t")
    merge = samples.merge(genotypes, left_on="IID", right_on="person_id", how="inner")
    
    sns.boxplot(x='geno', y='phenotype', data=merge, ax=axs[i])

    axs[i].set_title(file)  # Set title for each subplot
    #axs[i].scatter(merge['geno'], merge['phenotype']) # Example plot (replace with your actual plot)
    
# Hide any unused subplots (if there are less than 9 files)
for j in range(i + 1, len(axs)):
    axs[j].axis('off')  # Turn off axes for unused subplots

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
pcs = pd.read_csv("AOU_10_PCS.tsv", sep="\t")
ancestry = pd.read_csv("id_ancestry.tsv", sep="\t")
pc = pd.read_csv("platelet_count_new_phenocovar.csv")
eur = pd.read_csv("EUR_WHITE.csv")
not_afr = pd.read_csv("NOT_AFR_BLACK.csv")

In [ ]:
not_afr_set_diff_eur = not_afr[~not_afr['person_id'].isin(set(eur['person_id']))]

In [ ]:
not_afr_set_diff_eur

In [ ]:
aou_merge = pcs.merge(ancestry, left_on='IID', right_on='research_id', how='inner')
pc_merge = not_afr_set_diff_eur.merge(pc, on='person_id', how='inner')

In [ ]:
aou_merge['color'] = aou_merge['ancestry_pred'].apply(lambda x: 'eur' if x == 'eur' else 'other')

In [ ]:
aou_merge.loc[aou_merge['research_id'].isin(set(pc_merge['person_id'])), 'color'] = 'not_afr_not_eur'

In [ ]:
def plot_pc_pairs(df, label_col):
    pc_x = 'pc1'
    pc_y = 'pc2'
    colors = ['#1f77b4', '#7f7f7f', '#2ca02c']  # Blue, Green, Gray in hex codes
    sns.set_palette(colors)
    sns.scatterplot(data=df, x=df[pc_x], y=df[pc_y], hue=label_col)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    #plt.title(f'{pc_x} vs {pc_y}')
    plt.legend(title="Ancestry")

    plt.tight_layout()
    plt.show()
plot_pc_pairs(aou_merge, label_col='color')

In [ ]:
aou_merge[aou_merge['research_id']==1000104]

In [ ]:
ancestry_preds = pd.read_csv("ancestry_preds.tsv", sep="\t")

In [ ]:
#afr,amr,eas,eur,mid,sas
for i in range(6):
    ancestry_preds[f"p{i}"] = ancestry_preds['probabilities'].apply(lambda x: ast.literal_eval(x)[i])

In [ ]:
pcsubset_notafreur = ancestry_preds.merge(pc_merge, left_on='research_id', right_on='person_id',how='inner')

In [ ]:
plt.hist(pcsubset_notafreur['p3']+pcsubset_notafreur['p1'])
plt.xlabel("P(AMR)+P(EUR)")

In [ ]:
sns.countplot(x=pcsubset_notafreur['ancestry_pred_other'])


In [ ]:
pc_merge